In [1]:
import sys    
print('Running on Python version: {}'.format(sys.version))


Running on Python version: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]


In [2]:
import seaborn as sns
import os
%matplotlib inline


In [3]:
import pickle
try:
    import prettyplotlib as ppl
    from prettyplotlib import plt
    from prettyplotlib import brewer2mpl
except:
    try:
        !pip install prettyplotlib
        import prettyplotlib as ppl
        from prettyplotlib import plt
        from prettyplotlib import brewer2mpl
        
        green_purple = brewer2mpl.get_map('PRGn', 'diverging', 11).mpl_colormap

    except:
        import matplotlib.plot as plt
import numpy as np
import string



import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math as math
from pylab import rcParams
from neuronunit.optimization.results_analysis import make_report, min_max
with open('pre_grid_reports.p','rb') as f:#
    grid_results = pickle.load(f)

with open('pre_ga_reports.p','rb') as f:
    package = pickle.load(f)
pop = package[0]
print(pop[0].dtc.attrs.items())
history = package[4]
gen_vs_pop =  package[6]


    100% |████████████████████████████████| 706kB 6.2MB/s ta 0:00:011   32% |██████████▍                     | 225kB 4.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for prettyplotlib ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/ad/45/9fcfb9e97ecccc850d8b2fb20b8d60992bc6d7c8d9769ee989
Successfully built prettyplotlib
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


dict_items([('a', 0.27328841763197198), ('b', -2.5511245505753264e-09), ('vr', -71.213934167873902)])


In [4]:
attrs_ = [ list(p.dtc.attrs.keys()) for i,p in history.genealogy_history.items() ]
attrs = attrs_[0]
print(attrs)

scores_ = [ list(p.dtc.scores.keys()) for i,p in history.genealogy_history.items() ]
scores = scores_[0]
from collections import OrderedDict

urlDats = []
hi = [ (i,p) for i,p in history.genealogy_history.items() ]
pops_ = [ (i,p) for i,p in enumerate(pop) ]

sc = [ (i,p) for i,p in enumerate(grid_results) ]

import quantities as pq

def history_iter(mapped):
    i,p = mapped
    gene_contents = OrderedDict()
    #gene_contents['gene_number'] = i
    
    attrs = list(p.dtc.attrs.keys()) 
    scores = list(p.dtc.scores.keys()) 
    for a in attrs:
        gene_contents[a] = p.dtc.attrs[a]    
    scores0 = scores[0]
    for s in scores:
        gene_contents[s] = p.dtc.scores[s]
    gene_contents[str('total')] = sum(p.dtc.scores.values())
    for test in p.dtc.score.keys():
        if 'agreement' in p.dtc.score[test]:
            gene_contents['disagreement'] = p.dtc.score[test]['agreement']
        if 'observation' in p.dtc.score[test].keys() and 'prediction' in p.dtc.score[test].keys():
            boolean_means = bool('mean' in  p.dtc.score[test]['observation'].keys() and 'mean' in  p.dtc.score[test]['prediction'].keys())
            boolean_value = bool('value' in  p.dtc.score[test]['observation'].keys() and 'value' in p.dtc.score[test]['prediction'].keys())

            if boolean_means:
                gene_contents['disagreement'] = np.abs(p.dtc.score[test]['observation']['mean'] - p.dtc.score[test]['prediction']['mean'])
            if boolean_value:
                gene_contents['disagreement'] = np.abs(p.dtc.score[test]['observation']['value'] - p.dtc.score[test]['prediction']['value'])
                print(gene_contents['disagreement'])
    
    return gene_contents


    
def process_dics(contents):
    dfs = []
    for gene_contents in contents:
        # pandas Data frames are best data container for maths/stats, but steep learning curve.
        # Other exclusion criteria. Exclude reading levels above grade 100,
        # as this is most likely a problem with the metric algorithm, and or rubbish data in.

        if len(dfs) == 0:
            dfs = pd.DataFrame(pd.Series(gene_contents)).T
        dfs = pd.concat([ dfs, pd.DataFrame(pd.Series(gene_contents)).T ])
    return dfs

genes = list(map(history_iter,hi))    
dfg = process_dics(genes)

grids = list(map(history_iter,sc))    
dfs = process_dics(grids)

dfg = dfg.reset_index(drop=True)

# Set colormap equal to seaborns light green color palette
cm = sns.light_palette("green", as_cmap=True)
display(dfg.style.background_gradient(cmap=cm,subset=['total']))


['a', 'b', 'vr']
